In [ ]:
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import requests
import os

# Adjust configuration below
PREFIX_URL = 'https://www.etutor.pl'
DOWNLOAD = True
HTML_INPUT = 'C:\\Users\\lukas\Desktop\\Twoje słówka angielskie (8) - eTutor.html'
EXCEL_OUTPUT = 'sentences.xlsx'

# Download html page from eTutor (!important tab: "Podgląd" NIE "Elementy")
with open(HTML_INPUT, 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

if not os.path.exists('audio'):
    os.makedirs('audio')

sentences = []

learning_contents = soup.find('div', class_='learningcontents')
words = learning_contents.find_all('div', recursive=False)

for pharse in words:
    try:
        word = pharse.find('span', class_='phraseEntityLine').get_text(strip=True)
        translations = pharse.find('div', class_='hws phraseEntity', recursive=False).find_all(string=True, recursive=False)[1].replace('=', '').strip()
        audio_icon = pharse.find('span', class_='audioIcon')
        audio_url, audio_file = '', ''
        if audio_icon:
            audio_url = PREFIX_URL + pharse.find('span', class_='audioIcon')['data-audio-url']
            audio_file = audio_url.split('/')[-1]
        sentence = pharse.find('ul', class_='sentencesul')
        sentence_pl = ''
        sentence_eng = ''
        if sentence:
            sentence_eng = sentence.find_all(string=True)[1].replace('=', '').strip()
            sentence_pl = sentence.find('span', class_='sentenceTranslation').get_text(strip=True)

        diki_item = {
            'word': word,
            'translations': translations,
            'audio_url': audio_file,
            'sentence_pl': sentence_pl,
            'sentence_eng': sentence_eng
        }

        print(diki_item)
        sentences.append(diki_item)

        if DOWNLOAD and audio_url and (r:=requests.get(audio_url)).status_code != 404:
            print(f'Downloading {audio_file}')
            open(os.path.join("audio", audio_file), 'wb').write(r.content)
            sleep(0.1)
        
    except Exception as e:
        print(e)

pd.DataFrame(sentences).to_excel(EXCEL_OUTPUT, index=False)


{'word': 'washing powder', 'translations': 'proszek do prania', 'audio_url': 'washing_powder.mp3', 'sentence_pl': 'Ariel to mój ulubiony proszek do prania.', 'sentence_eng': 'Ariel is my favourite washing powder.'}
{'word': 'soap powder', 'translations': 'proszek mydlany, proszek do prania', 'audio_url': 'soap_powder.mp3', 'sentence_pl': '', 'sentence_eng': ''}
{'word': 'sleepless', 'translations': 'bezsenny (nie mogący zasnąć)', 'audio_url': 'sleepless.mp3', 'sentence_pl': '', 'sentence_eng': ''}
{'word': 'cut someone some slack', 'translations': 'dać komuś spokój, daj komuś trochę luzu', 'audio_url': 'cut_someone_some_slack.mp3', 'sentence_pl': 'Daj mu spokój, dopiero co zmarł mu ojciec.', 'sentence_eng': 'Cut him a little slack, his father just died.'}
{'word': 'take up the slack', 'translations': 'zastępować kogoś', 'audio_url': 'take_up_the_slack.mp3', 'sentence_pl': '', 'sentence_eng': ''}
{'word': 'slack season', 'translations': 'martwy sezon, zastój (np. w handlu)', 'audio_url'